In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
a=['D1','D31','D32','D4MEAT2','D4MEAT1','D4FISH2','D4FISH1','D4EGG1','D4BEAN1','D4VEG2','D4VEG1','D4SUGA2','D4TEA2','D4TEA1','D4GARL2','D4GARL1','D4MILK1','D4MILK2','D4NUT1','D4NUT2','D4ALGA1','D4ALGA2','D4VIT2','D4DRUG1','D4DRUG2','D6B']

In [16]:
filtered_df=pd.read_excel('dietAging.xlsx')
filtered_df

,rate,D1,D31,D32,D4MEAT2,D4MEAT1,D4FISH2,D4FISH1,D4EGG1,D4BEAN1,...,D4MILK1,D4MILK2,D4NUT1,D4NUT2,D4ALGA1,D4ALGA2,D4VIT2,D4DRUG1,D4DRUG2,D6B
0,-0.154726,3,3,2,2,2,2,5,2,2,...,2,5,3,5,2,2,5,5,5,5
1,-0.006561,4,4,1,5,5,3,2,2,2,...,1,5,5,5,3,5,5,5,5,1
2,-0.122017,3,2,2,1,2,2,2,1,2,...,1,5,4,2,4,2,5,5,5,5
3,-0.131271,4,3,1,2,3,2,3,1,2,...,4,5,5,1,3,3,5,5,5,1
4,0.048959,4,4,3,2,2,5,5,3,2,...,5,5,2,2,2,5,5,5,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5662,-0.284096,4,4,1,2,2,2,2,1,1,...,1,1,5,5,2,2,5,1,1,5
5663,-0.140972,1,4,1,2,2,4,4,2,2,...,3,3,5,5,4,4,4,3,3,1
5664,0.009188,1,4,1,2,3,4,5,1,5,...,3,1,5,5,4,5,1,3,5,5
5665,-0.016312,4,3,2,5,5,5,5,2,1,...,5,2,5,3,5,3,5,3,5,5


In [17]:
'''
columns_to_keep = ['age', 'kdm'] + a
filtered_df = pd[columns_to_keep]
filtered_df
'''

"\ncolumns_to_keep = ['age', 'kdm'] + a\nfiltered_df = pd[columns_to_keep]\nfiltered_df\n"

In [18]:
'''
filtered_df.insert(filtered_df.columns.get_loc('kdm') + 1, 'rate', (filtered_df['kdm'] - filtered_df['age']) / filtered_df['age'])
filtered_df
filtered_df.to_excel('dietAging.xlsx',index=False)
'''

"\nfiltered_df.insert(filtered_df.columns.get_loc('kdm') + 1, 'rate', (filtered_df['kdm'] - filtered_df['age']) / filtered_df['age'])\nfiltered_df\nfiltered_df.to_excel('dietAging.xlsx',index=False)\n"

In [19]:
#ANOVA
import statsmodels.api as sm
from statsmodels.formula.api import ols
import warnings
from tqdm import tqdm
from pprint import pprint

# 关闭所有警告
warnings.filterwarnings("ignore")

# 获取所有非rate的因素列
factors = [col for col in filtered_df.columns if col not in  ['rate','D1']]

# 遍历所有因素两两组合
interaction_effects = {}
for i in tqdm(range(len(factors))):
    for j in range(i + 1, len(factors)):
        factor1 = factors[i]
        factor2 = factors[j]
        
        # 构建双因素ANOVA模型
        model = ols(f'rate ~ C({factor1}) * C({factor2})', data=filtered_df).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)
        
        # 检查交互效应的显著性
        interaction_p_value = anova_table.loc[f'C({factor1}):C({factor2})', 'PR(>F)']
        interaction_effect_strength = anova_table.loc[f'C({factor1}):C({factor2})', 'sum_sq']

        if interaction_p_value < 0.05 and abs(interaction_effect_strength)>1:
            # 输出显著交互效应的组合及其强度
            interaction_effects[f"{factor1} 和 {factor2}"] = interaction_effect_strength
            
pprint(interaction_effects)


100%|██████████| 25/25 [00:09<00:00,  2.59it/s]

{'D4BEAN1 和 D4TEA1': 1.2218819760037058,
 'D4BEAN1 和 D4TEA2': 1.491335782185347,
 'D4EGG1 和 D4TEA1': 1.2117056831861799,
 'D4EGG1 和 D4TEA2': 1.3013277255705789,
 'D4FISH1 和 D4TEA1': 1.0637379311652433,
 'D4FISH1 和 D4TEA2': 1.2122393254123995,
 'D4MEAT1 和 D4TEA1': 1.1360428164939977,
 'D4MEAT1 和 D4TEA2': 1.6427547488790453,
 'D4MILK2 和 D4ALGA2': 1.0687274721752926,
 'D4NUT1 和 D4DRUG2': 1.191093347907527,
 'D4TEA1 和 D4ALGA1': 1.3314101027139675,
 'D4TEA1 和 D4ALGA2': 1.4720901976035685,
 'D4TEA1 和 D4MILK2': 1.4291392634045164,
 'D4TEA1 和 D4NUT1': 1.7232416065909208,
 'D4TEA1 和 D6B': 1.2686704630816559,
 'D4TEA2 和 D4ALGA1': 1.6429833693114284,
 'D4TEA2 和 D4ALGA2': 1.2171394446453416,
 'D4TEA2 和 D4MILK2': 1.718424695133248,
 'D4TEA2 和 D4NUT1': 1.0743922118976466,
 'D4TEA2 和 D6B': 1.5341129396361233,
 'D4VIT2 和 D4DRUG1': 1.1375900187496213}


In [20]:

filtered_df.rename(columns={
    'D1': '主食类型',
    'D31': '目前-新鲜水果',
    'D32': '目前-蔬菜',
    'D33': '您做饭主要用什么油脂',
    'D33A': '您主要使用的其他类型的油脂',
    'D34': '你的主要口味',
    'D4MEAT2': '目前-肉',
    'D4MEAT1': '60岁-肉',
    'D4FISH2': '目前-鱼',
    'D4FISH1': '60岁-鱼',
    'D4EGG2': '目前-鸡蛋',
    'D4EGG1': '60岁-鸡蛋',
    'D4BEAN2': '目前-豆类食品',
    'D4BEAN1': '60岁-豆类食品',
    'D4VEG2': '目前-腌菜',
    'D4VEG1': '60岁-腌菜',
    'D4SUGA2': '目前-糖',
    'D4SUGA1': '60岁-糖',
    'D4TEA2': '目前-茶',
    'D4TEA1': '60岁-茶',
    'D4GARL2': '目前-大蒜',
    'D4GARL1': '60岁-大蒜',
    'D4MILK1': '目前-奶制品',
    'D4MILK2': '60岁-奶制品',
    'D4NUT1': '目前-坚果制品',
    'D4NUT2': '60岁-坚果制品',
    'D4ALGA1': '目前-蘑菇或藻类',
    'D4ALGA2': '60岁-蘑菇或藻类',
    'D4VIT1': '目前-维生素 (a/c/e)',
    'D4VIT2': '60岁-维生素 (a/c/e)',
    'D4DRUG1': '目前-药用植物',
    'D4DRUG2': '60岁-药用植物',
    'D4A': '目前一般喝什么茶',
    'D4B': '60岁-茶',
    'D5': '目前-水',
    'D6A': '儿童-水来源',
    'D6B': '60岁-水来源',
    'D6C': '目前-水来源'
}, inplace=True)


In [21]:
'''
# 计算rate与age, kdm以外列的相关性
correlation_matrix = filtered_df.corr()

# 提取rate与其他列的相关性
rate_correlations = correlation_matrix['rate'].drop(['rate'])

# 找出与rate关系最密切的列
most_correlated_columns = rate_correlations[rate_correlations.abs() > 0.09].sort_values(ascending=False)

# 输出与rate关系最密切的列及其相关系数
print("与抗衰老关系最密切的列及其相关系数：")
print(most_correlated_columns)

# 计算列之间的相关性矩阵
column_correlations = filtered_df.drop(['rate'], axis=1).corr()

# 找出潜在的协同或拮抗作用
synergistic_columns = {}
antagonistic_columns = {}

for col in most_correlated_columns.index:
    synergistic_columns[col] = column_correlations[col][column_correlations[col] > 0.3].index.tolist()
    antagonistic_columns[col] = column_correlations[col][column_correlations[col] < -0.2].index.tolist()

# 输出潜在的协同和拮抗作用
print("潜在的协同作用列：")
for col, synergies in synergistic_columns.items():
    if synergies:
        print(f"{col}: {synergies}")
'''

与抗衰老关系最密切的列及其相关系数：
60岁-蘑菇或藻类          0.150444
60岁-维生素 (a/c/e)    0.147575
60岁-药用植物           0.144071
目前-蘑菇或藻类           0.122604
60岁-坚果制品           0.120587
60岁-豆类食品           0.116030
60岁-奶制品            0.113248
60岁-鱼              0.100244
60岁-鸡蛋             0.099272
目前-药用植物            0.094711
60岁-肉              0.091387
60岁-水来源           -0.094200
Name: rate, dtype: float64
潜在的协同作用列：
60岁-蘑菇或藻类: ['60岁-奶制品', '60岁-坚果制品', '目前-蘑菇或藻类', '60岁-蘑菇或藻类', '60岁-药用植物']
60岁-维生素 (a/c/e): ['60岁-奶制品', '60岁-维生素 (a/c/e)', '60岁-药用植物']
60岁-药用植物: ['60岁-蘑菇或藻类', '60岁-维生素 (a/c/e)', '目前-药用植物', '60岁-药用植物']
目前-蘑菇或藻类: ['目前-奶制品', '目前-坚果制品', '目前-蘑菇或藻类', '60岁-蘑菇或藻类', '目前-药用植物']
60岁-坚果制品: ['60岁-坚果制品', '60岁-蘑菇或藻类']
60岁-豆类食品: ['60岁-肉', '60岁-鱼', '60岁-鸡蛋', '60岁-豆类食品']
60岁-奶制品: ['60岁-鸡蛋', '60岁-奶制品', '60岁-蘑菇或藻类', '60岁-维生素 (a/c/e)']
60岁-鱼: ['60岁-肉', '目前-鱼', '60岁-鱼', '60岁-鸡蛋', '60岁-豆类食品']
60岁-鸡蛋: ['60岁-肉', '60岁-鱼', '60岁-鸡蛋', '60岁-豆类食品', '60岁-奶制品']
目前-药用植物: ['目前-蘑菇或藻类', '目前-药用植物', '60岁-药用植物']
60岁-肉: ['60岁-肉', '60岁-鱼', '60岁-

In [22]:
'''
print("潜在的拮抗作用列：")
for col, antagonisms in antagonistic_columns.items():
    if antagonisms:
        print(f"{col}: {antagonisms}")
'''        

潜在的拮抗作用列：
60岁-蘑菇或藻类: ['60岁-水来源']
60岁-奶制品: ['60岁-水来源']
60岁-鱼: ['60岁-水来源']
60岁-肉: ['60岁-水来源']
60岁-水来源: ['60岁-肉', '60岁-鱼', '60岁-奶制品', '60岁-蘑菇或藻类']


In [23]:
#rate_correlations['60岁_水来源']

In [24]:
'''
from sklearn.decomposition import FactorAnalysis

# 进行因子分析
fa = FactorAnalysis(n_components=2)
factors = fa.fit_transform(filtered_df[most_correlated_columns.index])

# 输出因子
print(factors)
'''

'\nfrom sklearn.decomposition import FactorAnalysis\n\n# 进行因子分析\nfa = FactorAnalysis(n_components=2)\nfactors = fa.fit_transform(filtered_df[most_correlated_columns.index])\n\n# 输出因子\nprint(factors)\n'

In [25]:
'''
from sklearn.cluster import KMeans
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# 提取rate和其他非kdm, age的列
columns_of_interest = [col for col in filtered_df.columns if col not in ['kdm', 'age']]
data_for_clustering = filtered_df[['rate'] + columns_of_interest]

# 进行多元聚类
kmeans = KMeans(n_clusters=3, random_state=0).fit(data_for_clustering)

# 将聚类结果添加到原数据框中
filtered_df['cluster'] = kmeans.labels_

# 输出聚类结果
print("聚类结果：")
print(filtered_df[['rate', 'cluster'] + columns_of_interest].head())

# 使用PCA进行降维以便可视化
pca = PCA(n_components=2)
principal_components = pca.fit_transform(data_for_clustering)

# 创建一个新的DataFrame来存储PCA结果和聚类标签
pca_df = pd.DataFrame(data=principal_components, columns=['principal_component_1', 'principal_component_2'])
pca_df['cluster'] = kmeans.labels_

# 可视化聚类结果
plt.figure(figsize=(10, 6))
for cluster in pca_df['cluster'].unique():
    cluster_data = pca_df[pca_df['cluster'] == cluster]
    plt.scatter(cluster_data['principal_component_1'], cluster_data['principal_component_2'], label=f'Cluster {cluster}')
plt.title('聚类结果的PCA可视化')
plt.xlabel('主成分1')
plt.ylabel('主成分2')
plt.legend()
plt.show()
'''

'\nfrom sklearn.cluster import KMeans\nimport pandas as pd\nimport matplotlib.pyplot as plt\nfrom sklearn.decomposition import PCA\n\n# 提取rate和其他非kdm, age的列\ncolumns_of_interest = [col for col in filtered_df.columns if col not in [\'kdm\', \'age\']]\ndata_for_clustering = filtered_df[[\'rate\'] + columns_of_interest]\n\n# 进行多元聚类\nkmeans = KMeans(n_clusters=3, random_state=0).fit(data_for_clustering)\n\n# 将聚类结果添加到原数据框中\nfiltered_df[\'cluster\'] = kmeans.labels_\n\n# 输出聚类结果\nprint("聚类结果：")\nprint(filtered_df[[\'rate\', \'cluster\'] + columns_of_interest].head())\n\n# 使用PCA进行降维以便可视化\npca = PCA(n_components=2)\nprincipal_components = pca.fit_transform(data_for_clustering)\n\n# 创建一个新的DataFrame来存储PCA结果和聚类标签\npca_df = pd.DataFrame(data=principal_components, columns=[\'principal_component_1\', \'principal_component_2\'])\npca_df[\'cluster\'] = kmeans.labels_\n\n# 可视化聚类结果\nplt.figure(figsize=(10, 6))\nfor cluster in pca_df[\'cluster\'].unique():\n    cluster_data = pca_df[pca_df[\'cluster\'] ==

In [26]:
#多元变量回归
import pandas as pd
import statsmodels.api as sm
import itertools

# 假设你有一个DataFrame 'df' 包含 rate 和其他变量
df = filtered_df.copy()

# 获取所有列名，排除因变量 'rate' 以及 'age' 和 'kdm'
columns = [col for col in df.columns if col not in ['rate', 'age', 'kdm']]


# 生成所有列之间的交互项
for col1, col2 in itertools.combinations(columns, 2):
    df[f'{col1}_{col2}'] = df[col1] * df[col2]

# 定义自变量和因变量
X = df[[col for col in df.columns if col not in ['rate', 'age', 'kdm']]]
y = df['rate']

# 添加常数项
X = sm.add_constant(X)

# 构建回归模型
model = sm.OLS(y, X).fit()

# 输出coefficient最大的十组数据
significant_interactions = model.pvalues[model.pvalues < 0.05].index
coefficients = model.params[significant_interactions].drop('const').sort_values(ascending=False).head(10)
print("coefficient最大的十组数据：")
for interaction, coefficient in coefficients.items():
    print(f"{interaction}: {coefficient}")

# 输出coefficient非nan的最小的十组数据
non_nan_coefficients = model.params.dropna().drop('const').sort_values().head(10)
print("coefficient非nan的最小的十组数据：")
for interaction, coefficient in non_nan_coefficients.items():
    print(f"{interaction}: {coefficient}")

coefficient最大的十组数据：
目前-腌菜: 0.05912559711366786
60岁-药用植物: 0.05731268479530192
60岁-鱼: 0.05528099479243026
60岁-维生素 (a/c/e): 0.042423029303989244
60岁-奶制品: 0.03693088950558287
60岁-坚果制品_60岁-药用植物: 0.010736600173250689
目前-蘑菇或藻类_60岁-药用植物: 0.007813637196177284
目前-坚果制品_60岁-维生素 (a/c/e): 0.0068423509172715865
目前-鱼_60岁-维生素 (a/c/e): 0.006340339110241532
目前-鱼_目前-药用植物: 0.005692960393844614
coefficient非nan的最小的十组数据：
目前-鱼: -0.056692485644489934
60岁-茶: -0.0556529272312966
60岁-大蒜: -0.05412031603334356
目前-肉: -0.033079367784522626
目前-糖: -0.025093849022244243
目前-蔬菜: -0.021137967165505363
60岁-肉: -0.019046882022281712
目前-茶: -0.014540041376551358
60岁-鸡蛋: -0.01337892194591453
60岁-腌菜: -0.013331708986652892


In [31]:
# 计算相关性矩阵
correlation_matrix = df.corr()

# 提取因变量 'rate' 与其他自变量的相关性
rate_correlations = correlation_matrix['rate'].drop(['rate'])

# 输出显著的相关性
significant_correlations = rate_correlations[rate_correlations.abs() > 0.15]
print("显著的自变量（及其组合）与因变量 'rate' 的相关性：")
print(significant_correlations)

显著的自变量（及其组合）与因变量 'rate' 的相关性：
60岁-蘑菇或藻类                    0.150444
60岁-鱼_目前-蘑菇或藻类               0.152012
60岁-豆类食品_目前-蘑菇或藻类            0.155952
60岁-豆类食品_60岁-蘑菇或藻类           0.152569
60岁-坚果制品_目前-蘑菇或藻类            0.151237
60岁-坚果制品_60岁-蘑菇或藻类           0.150764
60岁-坚果制品_60岁-药用植物            0.151065
目前-蘑菇或藻类_60岁-蘑菇或藻类           0.160846
目前-蘑菇或藻类_60岁-维生素 (a/c/e)     0.155965
目前-蘑菇或藻类_60岁-药用植物            0.155084
60岁-蘑菇或藻类_60岁-维生素 (a/c/e)    0.173680
60岁-蘑菇或藻类_目前-药用植物            0.165522
60岁-蘑菇或藻类_60岁-药用植物           0.169883
60岁-维生素 (a/c/e)_60岁-药用植物     0.166412
Name: rate, dtype: float64


In [28]:
'''
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 定义自变量和因变量
X = df[[col for col in df.columns if col not in ['rate', 'age', 'kdm']]]
y = df['rate']

# 分割数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 构建梯度提升树模型
gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gbr.fit(X_train, y_train)

# 预测测试集
y_pred = gbr.predict(X_test)

# 计算均方误差
mse = mean_squared_error(y_test, y_pred)
print(f"均方误差: {mse}")

# 输出特征重要性
feature_importances = gbr.feature_importances_
important_features = pd.Series(feature_importances, index=X.columns).sort_values(ascending=False)
print("特征重要性：")
print("最大的10个特征值：")
print(important_features.head(10))
'''

'\nfrom sklearn.ensemble import GradientBoostingRegressor\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.metrics import mean_squared_error\n\n# 定义自变量和因变量\nX = df[[col for col in df.columns if col not in [\'rate\', \'age\', \'kdm\']]]\ny = df[\'rate\']\n\n# 分割数据集为训练集和测试集\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# 构建梯度提升树模型\ngbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)\ngbr.fit(X_train, y_train)\n\n# 预测测试集\ny_pred = gbr.predict(X_test)\n\n# 计算均方误差\nmse = mean_squared_error(y_test, y_pred)\nprint(f"均方误差: {mse}")\n\n# 输出特征重要性\nfeature_importances = gbr.feature_importances_\nimportant_features = pd.Series(feature_importances, index=X.columns).sort_values(ascending=False)\nprint("特征重要性：")\nprint("最大的10个特征值：")\nprint(important_features.head(10))\n'

In [29]:
'''
from sklearn.ensemble import RandomForestRegressor

# 构建随机森林回归模型
rfr = RandomForestRegressor(n_estimators=100, random_state=42)
rfr.fit(X_train, y_train)

# 预测测试集
y_pred_rfr = rfr.predict(X_test)

# 计算均方误差
mse_rfr = mean_squared_error(y_test, y_pred_rfr)
print(f"随机森林回归的均方误差: {mse_rfr}")

# 输出特征重要性
feature_importances_rfr = rfr.feature_importances_
important_features_rfr = pd.Series(feature_importances_rfr, index=X.columns).sort_values(ascending=False)
print("随机森林回归的特征重要性：")
print("最大的10个特征值：")
print(important_features_rfr.head(10))
'''

'\nfrom sklearn.ensemble import RandomForestRegressor\n\n# 构建随机森林回归模型\nrfr = RandomForestRegressor(n_estimators=100, random_state=42)\nrfr.fit(X_train, y_train)\n\n# 预测测试集\ny_pred_rfr = rfr.predict(X_test)\n\n# 计算均方误差\nmse_rfr = mean_squared_error(y_test, y_pred_rfr)\nprint(f"随机森林回归的均方误差: {mse_rfr}")\n\n# 输出特征重要性\nfeature_importances_rfr = rfr.feature_importances_\nimportant_features_rfr = pd.Series(feature_importances_rfr, index=X.columns).sort_values(ascending=False)\nprint("随机森林回归的特征重要性：")\nprint("最大的10个特征值：")\nprint(important_features_rfr.head(10))\n'